In [1]:
import pandas as pd
import tensorflow as tf
import scipy as sc
from scipy.spatial import distance
import cv2
import numpy as np
import threading
import time
import datetime
from multiprocessing import Queue
import os
import json
from base64 import b64encode
from typing import (
    Any,
    List,
    Tuple,
)

import align.detect_face    # type: ignore
import facenet              # type: ignore

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
#base_path = "/mnt/d/nna/"
base_path = "d:/nna/"
minimum_size_of_face = 60

frames = Queue(1)
faces = Queue(1)

STEP = 4

In [3]:
data_path = base_path + "data/"
facenet_path = base_path + "node-facenet/"
model_path = facenet_path + "models/"

In [4]:
class MtcnnBridge():
    """
    MTCNN Face Alignment
    """
    def __init__(self) -> None:
        self.graph = self.session = None            # type: Any
        self.pnet = self.rnet = self.onet = None    # type: Any

        self.minsize = minimum_size_of_face # minimum size of face
        self.threshold = [0.6, 0.7, 0.7]    # three steps's threshold
        self.factor = 0.709                 # scale factor

    def init(self) -> None:
        """ doc """

        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                self.pnet, self.rnet, self.onet = \
                    align.detect_face.create_mtcnn(self.session, None)

    def align(
            self,
            image: np.ndarray,
    ) -> Tuple[List[Any], List[Any]]:
        """ doc """

        bounding_boxes, landmarks = align.detect_face.detect_face(
            image,   # get rid of alpha channel(if any)
            self.minsize,
            self.pnet,
            self.rnet,
            self.onet,
            self.threshold,
            self.factor,
        )

        return bounding_boxes.tolist(), landmarks.tolist()

In [5]:
class FacenetBridge(object):
    """
    Bridge of Facenet
    """
    FACENET_MODEL = None   # type: str

    def __init__(self) -> None:
        self.graph = self.session = None        # type: Any

        self.placeholder_input = None           # type: Any
        self.placeholder_phase_train = None     # type: Any
        self.placeholder_embeddings = None      # type: Any

        self.FACENET_MODEL = FacenetBridge.get_model_path()

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                model_dir = os.path.expanduser(self.FACENET_MODEL)
                meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
                saver = tf.train.import_meta_graph(
                    os.path.join(model_dir, meta_file),
                )
                saver.restore(
                    tf.get_default_session(),
                    os.path.join(model_dir, ckpt_file),
                )
                # facenet.load_model(self.FACENET_MODEL)

        self.placeholder_input = self.graph.get_tensor_by_name('input:0')
        self.placeholder_phase_train = \
            self.graph.get_tensor_by_name('phase_train:0')
        self.placeholder_embeddings = \
            self.graph.get_tensor_by_name('embeddings:0')

    @staticmethod
    def get_model_path() -> str:
        return model_path    
    
    def embeddings(
            self,
            images: np.ndarray
    ) -> np.ndarray:
        """
        Get embeddings
        """

        slices = np.empty((len(images), 160, 160, 3))
        for i in range(len(images)):
            slices[i,:,:,:] = facenet.prewhiten(images[i])
            
        feed_dict = {
            self.placeholder_input:         slices,
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        #print("images " + str(len(images)))
        #print("emb " + str(len(embeddings)))
        
        # Return the only row
        return embeddings

In [6]:
class MtcnnMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.initNN()

    def initNN(self):
        self.mtcnn = MtcnnBridge()
        self.mtcnn.init()

    def run(self):
        print("Mtcnn thread: run")
        global frames
        global faces
        
        while True:
            #if(not frames.empty()):
                #remove extra faces
                #if (faces.full()): #skip oweloaded faces
                    #faces.get()
                
                #if frames.full(): #skip owerloaded frames
                    #frames.get()
                    
                frame = frames.get() 
                
                a = datetime.datetime.now()
                areas = self.mtcnn.align(frame[1])[0]                
                areasCount = len(areas)
                b = datetime.datetime.now()
                #print("Mtcnn align: " + str(b - a))     

                if (not areasCount == 0):
                    a = datetime.datetime.now()
                    
                    fullSize = frame[0]
                    height = np.size(fullSize, 0)
                    width = np.size(fullSize, 1)

                    positions = np.empty((areasCount, 4))
                    embeddings = np.empty((areasCount, 160, 160, 3))

                    #print(str(areasCount))
                    for i in range(areasCount):
                        pos = areas[i]
                        x = max(int(pos[0] * STEP), 0)
                        y = max(int(pos[1] * STEP), 0)
                        w = min(int(pos[2] * STEP), width)
                        h = min(int(pos[3] * STEP), height)
                        
                        positions[i,:] = [x,y,w,h] 
                        
                        img = fullSize[y:h, x:w, 0:3]
                        
                        
                        img = cv2.resize(img, dsize=(160, 160), interpolation=cv2.INTER_CUBIC)  
                        embeddings[i,:,:,:] = img 
                        
                        #print(positions[i,:])

                    if (not faces.full()):
                        faces.put([positions, embeddings])
                        
                    b = datetime.datetime.now()
                    #print("Mtcnn resizing: " + str(b - a))        

            #else:
                #print("Mtcnn Idle")
                time.sleep(0.001)
                    

In [7]:
class FacenetMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.user = []
        self.distances = []        
        self.loadUsers()
        self.initNN()

    def loadUsers(self):
        dataFile = "./data.json"
        file = open(dataFile, "r")
        self.users = json.loads(file.read())
        
    def initNN(self):
        self.face = FacenetBridge()
        self.face.init()

    def trackSessions(self, emb):
        a = datetime.datetime.now()
        nearDistances = []
        for user in self.users:
            userEmb = user['embedding']        
            userName = user['name']

            dist = distance.euclidean(userEmb, emb)
            if dist < 1.0:
                nearDistances.append((userName, dist))
                
        if(len(nearDistances) > 0):
            newlist = sorted(nearDistances, key=lambda x: x[1]) 

            print(newlist[0][0] + " - " + str(newlist[0][1]))
            
            self.distances.append(newlist[0][1])
        b = datetime.datetime.now()
        #print("Tracking sessions: " + str(b - a))
        
    def run(self):
        print("Facenet thread: run")
        
        global faces
        
        while True:
            if(not faces.empty()):
         
                #if faces.full(): # skip owerloaded faces
                    #faces.get()
                    
                facesOnScreen = faces.get()
                
                a = datetime.datetime.now()
                
                positions = facesOnScreen[0]
                images = facesOnScreen[1]
                                
                embeddings = self.face.embeddings(images)
                
                for emb in embeddings:                    
                    self.trackSessions(emb)      
                    
                b = datetime.datetime.now()   
                #print("Facenet: " + str(b - a))
            #else:
                #print("Facenet Idle")
            time.sleep(0.005)


In [8]:
class ImageGrabber(threading.Thread):
    def __init__(self, ID):
        threading.Thread.__init__(self)
        self.ID=ID
        self.restartGrabber()

    def restartGrabber(self):
        if(hasattr(self, "cam")):
            self.cam.release()
            time.sleep(0.05)
            
        self.cam=cv2.VideoCapture(self.ID)
        time.sleep(0.05)
        
        
    def run(self):
        print("Image grabber thread: run")
        
        global frames
        
        while True:
            #print('start')
            
            ret, frame=self.cam.read()
            if (ret):
                if(not frames.empty()):
                    time.sleep(0.001)
                    continue
                    
                # remove extra frames
                
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                height = np.size(frame, 0)
                width = np.size(frame, 1)
                
                #print("h: " + str(height) + " w: "+ str(width))
                frame2 = cv2.resize(frame, dsize=(int(width / STEP), int(height / STEP)), interpolation=cv2.INTER_CUBIC)

                
                if (frames.empty()):
                    a = datetime.datetime.now()
                    frames.put([frame, frame2])
                    b = datetime.datetime.now()
                    #print("Grabber put: " + str(b - a))

                    
            else:
                print('Grabber Idle')
                self.restartGrabber()
                
            time.sleep(0.001)

In [ ]:
mtcnnCount = 2
facenetCount = 2

mtcnnInstances = []
facenetInstances = []

grabberInstance = ImageGrabber(0)
#grabberInstance = ImageGrabber("rtsp://guest:sup3rGu3st@@10.10.101.156/ch1_1.h264")

for i in range(mtcnnCount):
    mtcnnInstances.append(MtcnnMain())
    
for i in range(facenetCount):
    facenetInstances.append(FacenetMain())

grabberInstance.start()

for mtcnn in mtcnnInstances:
    mtcnn.start()

for fn in facenetInstances:
    fn.start()
    fn.join()

for mtcnn in mtcnnInstances:
    mtcnn.join()

grabberInstance.join()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from d:/nna/node-facenet/models/model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from d:/nna/node-facenet/models/model-20170512-110547.ckpt-250000
Image grabber thread: runMtcnn thread: run
Mtcnn thread: run

Facenet thread: run
Oleksii Makarov - 0.7532024333311369
Oleksii Makarov - 0.7113711699798217
Oleksii Makarov - 0.7447874133049478
Oleksii Makarov - 0.7464252295296199
Oleksii Makarov - 0.7718431973921481
Oleksii Makarov - 0.772354340261572
Oleksii Makarov - 0.7977225374038808
Oleksii Makarov - 0.7077675991539261
Oleksii Makarov - 0.762694421179664
Oleksii Makarov - 0.7126136171695903
Oleksii Makarov - 0.7595327143497651
Oleksii Makarov - 0.7540048491452408
Oleksii Makarov - 0.7068101451704849
Oleksii Makarov - 0.7114856929290262
Olek